In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
test = pd.read_csv('test.csv')

In [3]:
test

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,saledate
0,2005,Cadillac,CTS,Base,Sedan,automatic,1g6dp567450124779,ca,2.7,116970.0,silver,black,lexus of stevens creek,Wed Jan 14 2015 04:30:00 GMT-0800 (PST)
1,2014,GMC,Savana Cargo,2500,Van,NaN,1gtw7fca7e1902207,pa,4.4,6286.0,white,gray,u-haul,Fri Feb 27 2015 01:00:00 GMT-0800 (PST)
2,2013,Nissan,Murano,S,SUV,automatic,jn8az1mw6dw303497,oh,4.6,11831.0,gray,black,nissan-infiniti lt,Tue Feb 24 2015 01:30:00 GMT-0800 (PST)
3,2013,Chevrolet,Impala,LS Fleet,Sedan,automatic,2g1wf5e34d1160703,fl,2.3,57105.0,silver,black,onemain rem/auto club of miami inc dba north dad,Fri Mar 06 2015 02:00:00 GMT-0800 (PST)
4,2013,Nissan,Titan,SV,Crew Cab,automatic,1n6aa0ec3dn301209,tn,2.9,31083.0,black,black,nissan north america inc.,Wed Jun 03 2015 03:30:00 GMT-0700 (PDT)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110053,2011,Jeep,Wrangler,Sport,SUV,automatic,1j4aa2d17bl584330,ca,3.8,66085.0,orange,black,dtg operations inc,Thu Jan 15 2015 03:30:00 GMT-0800 (PST)
110054,2014,Lexus,IS 250,Base,sedan,automatic,jthbf1d23e5007526,fl,4.8,17588.0,gray,black,lexus financial services,Mon Jun 15 2015 11:00:00 GMT-0700 (PDT)
110055,2004,Nissan,Maxima,3.5 SL,Sedan,automatic,1n4ba41e54c831950,va,2.9,124036.0,silver,gray,shirlie slack mitsubishi,Thu Feb 05 2015 01:35:00 GMT-0800 (PST)
110056,2013,Nissan,370Z,Base,coupe,manual,jn1az4eh3dm382431,tx,2.2,27169.0,—,black,nissan motor acceptance corporation,Wed Jun 17 2015 03:30:00 GMT-0700 (PDT)


In [4]:
train = pd.read_csv('train.csv')
train

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440231,2013,Kia,Optima,EX Hybrid,Sedan,automatic,knagm4ad4d5057805,ca,4.5,20434.0,white,beige,"kia motors america, inc",16000,Tue Jan 20 2015 04:30:00 GMT-0800 (PST)
440232,2014,MINI,Cooper Countryman,S ALL4,Wagon,automatic,wmwzc5c55ewp36517,wi,2.9,14675.0,white,black,bmw mini financial services,24250,Wed Feb 11 2015 02:00:00 GMT-0800 (PST)
440233,2008,BMW,6 Series,650i,Coupe,automatic,wbaea53588cv90110,ca,3.7,102552.0,gray,black,studio city auto group,16750,Tue Mar 03 2015 04:30:00 GMT-0800 (PST)
440234,2014,Ram,2500,SLT,Crew Cab,automatic,3c6ur5dl2eg243568,tx,3.0,29367.0,white,gray,express car & truck rental,33000,Wed Jan 14 2015 02:05:00 GMT-0800 (PST)


In [5]:
example = pd.read_csv('sample_submission.csv')
example

,vin,sellingprice
0,1g6dp567450124779,13592.209588
1,1gtw7fca7e1902207,13592.209588
2,jn8az1mw6dw303497,13592.209588
3,2g1wf5e34d1160703,13592.209588
4,1n6aa0ec3dn301209,13592.209588
...,...,...
110053,1j4aa2d17bl584330,13592.209588
110054,jthbf1d23e5007526,13592.209588
110055,1n4ba41e54c831950,13592.209588
110056,jn1az4eh3dm382431,13592.209588


In [6]:
example.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   vin           110058 non-null  object 
 1   sellingprice  110058 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.7+ MB


In [7]:
test.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'saledate'],
      dtype='object')

In [8]:
train.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'sellingprice',
       'saledate'],
      dtype='object')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


In [10]:
train.duplicated().sum()

0

In [11]:
train['year'].drop_duplicates().sort_values()

250627    1982
106264    1983
191684    1984
31280     1985
31287     1986
7349      1987
6666      1988
3756      1989
8779      1990
4097      1991
10128     1992
2479      1993
475       1994
934       1995
166       1996
414       1997
127       1998
89        1999
8         2000
86        2001
76        2002
3         2003
17        2004
9         2005
13        2006
4         2007
19        2008
58        2009
6         2010
0         2011
2         2012
5         2013
1         2014
62        2015
Name: year, dtype: int64

In [12]:
train

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440231,2013,Kia,Optima,EX Hybrid,Sedan,automatic,knagm4ad4d5057805,ca,4.5,20434.0,white,beige,"kia motors america, inc",16000,Tue Jan 20 2015 04:30:00 GMT-0800 (PST)
440232,2014,MINI,Cooper Countryman,S ALL4,Wagon,automatic,wmwzc5c55ewp36517,wi,2.9,14675.0,white,black,bmw mini financial services,24250,Wed Feb 11 2015 02:00:00 GMT-0800 (PST)
440233,2008,BMW,6 Series,650i,Coupe,automatic,wbaea53588cv90110,ca,3.7,102552.0,gray,black,studio city auto group,16750,Tue Mar 03 2015 04:30:00 GMT-0800 (PST)
440234,2014,Ram,2500,SLT,Crew Cab,automatic,3c6ur5dl2eg243568,tx,3.0,29367.0,white,gray,express car & truck rental,33000,Wed Jan 14 2015 02:05:00 GMT-0800 (PST)


In [13]:
median_condition = train['condition'].median()
median_odometer = train['odometer'].median()

In [14]:
train_mod = train
train_mod['condition']=train_mod['condition'].fillna(median_condition)
train_mod['odometer']=train_mod['odometer'].fillna(median_odometer)
train_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     440236 non-null  float64
 9   odometer      440236 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


In [15]:
train_mod['make'].unique()

array(['Ford', 'Nissan', 'HUMMER', 'Lincoln', 'pontiac', 'Hyundai', nan,
       'Buick', 'Chevrolet', 'Honda', 'Acura', 'Cadillac', 'GMC', 'Saab',
       'Dodge', 'Mercedes-Benz', 'Toyota', 'Volkswagen', 'BMW',
       'Infiniti', 'Chrysler', 'Kia', 'Jaguar', 'Subaru', 'Jeep', 'Lexus',
       'mercedes', 'Scion', 'FIAT', 'Suzuki', 'Mazda', 'Volvo', 'Audi',
       'MINI', 'Isuzu', 'Mitsubishi', 'smart', 'Pontiac', 'Porsche',
       'subaru', 'ford', 'Land Rover', 'chrysler', 'Saturn', 'mazda',
       'dodge', 'Ram', 'Oldsmobile', 'hyundai', 'Mercury', 'Bentley',
       'toyota', 'lincoln', 'Fisker', 'nissan', 'chevrolet', 'honda',
       'porsche', 'mitsubishi', 'lexus', 'bmw', 'Maserati', 'acura',
       'jeep', 'mercury', 'Tesla', 'landrover', 'vw', 'cadillac', 'buick',
       'gmc truck', 'land rover', 'volkswagen', 'Rolls-Royce', 'audi',
       'Ferrari', 'suzuki', 'Plymouth', 'oldsmobile', 'Lamborghini',
       'gmc', 'Geo', 'ford truck', 'Aston Martin', 'plymouth', 'Daewoo',
      

In [16]:
lower_columns = ['make', 'model', 'trim', 'body', 'transmission', 'vin','state', 'color', 'interior', 'seller']
for i in lower_columns:
    train_mod[i]=train_mod[i].str.lower()
train_mod['make'].unique()

array(['ford', 'nissan', 'hummer', 'lincoln', 'pontiac', 'hyundai', nan,
       'buick', 'chevrolet', 'honda', 'acura', 'cadillac', 'gmc', 'saab',
       'dodge', 'mercedes-benz', 'toyota', 'volkswagen', 'bmw',
       'infiniti', 'chrysler', 'kia', 'jaguar', 'subaru', 'jeep', 'lexus',
       'mercedes', 'scion', 'fiat', 'suzuki', 'mazda', 'volvo', 'audi',
       'mini', 'isuzu', 'mitsubishi', 'smart', 'porsche', 'land rover',
       'saturn', 'ram', 'oldsmobile', 'mercury', 'bentley', 'fisker',
       'maserati', 'tesla', 'landrover', 'vw', 'gmc truck', 'rolls-royce',
       'ferrari', 'plymouth', 'lamborghini', 'geo', 'ford truck',
       'aston martin', 'daewoo', 'mercedes-b', 'dodge tk', 'mazda tk',
       'dot', 'airstream'], dtype=object)

In [17]:
train_mod['make'].unique()

array(['ford', 'nissan', 'hummer', 'lincoln', 'pontiac', 'hyundai', nan,
       'buick', 'chevrolet', 'honda', 'acura', 'cadillac', 'gmc', 'saab',
       'dodge', 'mercedes-benz', 'toyota', 'volkswagen', 'bmw',
       'infiniti', 'chrysler', 'kia', 'jaguar', 'subaru', 'jeep', 'lexus',
       'mercedes', 'scion', 'fiat', 'suzuki', 'mazda', 'volvo', 'audi',
       'mini', 'isuzu', 'mitsubishi', 'smart', 'porsche', 'land rover',
       'saturn', 'ram', 'oldsmobile', 'mercury', 'bentley', 'fisker',
       'maserati', 'tesla', 'landrover', 'vw', 'gmc truck', 'rolls-royce',
       'ferrari', 'plymouth', 'lamborghini', 'geo', 'ford truck',
       'aston martin', 'daewoo', 'mercedes-b', 'dodge tk', 'mazda tk',
       'dot', 'airstream'], dtype=object)

In [18]:
train_mod['model'].unique()

array(['edge', 'fusion', 'sentra', 'h2', 'mkz', 'g6', 'escape', 'elantra',
       'freestyle', nan, 'lucerne', 'windstar', 'silverado 1500',
       'murano', 'equinox', 'accord', 'civic', 'mdx', 'cts', 'taurus',
       'srx', 'yukon', '9-3', 'explorer', 'f-150', 'charger', 'armada',
       'gl-class', 'avalon', 'tahoe', 'malibu', 'passat', 'camry', 'flex',
       '3 series', 'q50', 'altima', 'mustang', '300', 'sonata',
       'envoy xl', 'rogue', 'accent', 'maxima', 'forte', 'xf', 'outback',
       'grand caravan', 'e-class', 'cherokee', 'impala', 'optima',
       '5 series', 'expedition', 'is 250', 'mks', '200', 'veloster',
       'golf', 'rav4', 'e300dt', 'five hundred', 'g coupe',
       'grand cherokee', 'x-type', 'g convertible', 'tacoma', 'xa',
       'g sedan', 'tl', 'liberty', 'soul', '500l', 'town and country',
       'hhr', 'wrangler', 'reno', 'suburban', 'pt cruiser', 'gx 460',
       'sienna', 'sl-class', 'envoy', 'patriot', 'e-series van', 'versa',
       'aspen', 'm', 'co

In [19]:
train_mod['trim'].unique()

array(['sel', 'se', '2.0 sl', ..., '2.5 x l.l.bean edition',
       '3500 high roof 140 wb', '4x4 v6 xlt sport'], dtype=object)

In [20]:
train_mod['body'].unique()

array(['suv', 'sedan', nan, 'wagon', 'minivan', 'extended cab',
       'regular cab', 'coupe', 'supercrew', 'convertible', 'crew cab',
       'supercab', 'hatchback', 'g coupe', 'g convertible', 'access cab',
       'g sedan', 'e-series van', 'quad cab', 'tsx sport wagon', 'van',
       'cts coupe', 'koup', 'king cab', 'double cab', 'elantra coupe',
       'crewmax cab', 'genesis coupe', 'g37 coupe', 'club cab',
       'beetle convertible', 'mega cab', 'regular-cab', 'xtracab',
       'q60 coupe', 'promaster cargo van', 'cts-v coupe', 'cts wagon',
       'cab plus 4', 'g37 convertible', 'transit van', 'q60 convertible',
       'cab plus', 'ram van', 'granturismo convertible', 'cts-v wagon'],
      dtype=object)

In [21]:
train_mod['transmission'].unique()

array(['automatic', nan, 'manual'], dtype=object)

In [22]:
train_mod['color'].unique()

array(['black', 'gold', 'silver', 'blue', 'white', 'gray', '—', 'red',
       'brown', 'green', 'beige', 'orange', nan, 'off-white', 'burgundy',
       'yellow', 'charcoal', 'purple', 'turquoise', 'lime', 'pink'],
      dtype=object)

In [23]:
train_mod['interior'].unique()

array(['black', 'beige', 'gray', 'tan', 'brown', '—', 'off-white', nan,
       'blue', 'white', 'silver', 'red', 'green', 'gold', 'purple',
       'orange', 'burgundy', 'yellow'], dtype=object)

In [24]:
train_mod['seller'].unique()

array(['santander consumer', 'ars/avis budget group',
       'nissan-infiniti lt', ..., 'autostar enterprises',
       'kocourek nissan', 'studio city auto group'], dtype=object)

In [25]:
fillna_columns=['make', 'model', 'trim', 'body', 'transmission', 'color', 'interior']
for i in fillna_columns:
    train_mod[i]=train_mod[i].fillna('—')
train_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          440236 non-null  object 
 2   model         440236 non-null  object 
 3   trim          440236 non-null  object 
 4   body          440236 non-null  object 
 5   transmission  440236 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     440236 non-null  float64
 9   odometer      440236 non-null  float64
 10  color         440236 non-null  object 
 11  interior      440236 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


In [26]:
train_mod.head(5)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,ford,edge,sel,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,ford,fusion,se,sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,nissan,sentra,2.0 sl,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,hummer,h2,base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,ford,fusion,sel,sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)


In [27]:
train_mod['saledate']=pd.to_datetime(train_mod['saledate'], format='%a %b %d %Y', exact=False)

In [28]:
train_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   year          440236 non-null  int64         
 1   make          440236 non-null  object        
 2   model         440236 non-null  object        
 3   trim          440236 non-null  object        
 4   body          440236 non-null  object        
 5   transmission  440236 non-null  object        
 6   vin           440236 non-null  object        
 7   state         440236 non-null  object        
 8   condition     440236 non-null  float64       
 9   odometer      440236 non-null  float64       
 10  color         440236 non-null  object        
 11  interior      440236 non-null  object        
 12  seller        440236 non-null  object        
 13  sellingprice  440236 non-null  int64         
 14  saledate      440236 non-null  datetime64[ns]
dtypes: datetime64[ns]

In [29]:
train_mod.head(5)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,ford,edge,sel,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,2015-06-02
1,2014,ford,fusion,se,sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,2015-02-25
2,2012,nissan,sentra,2.0 sl,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,2015-06-10
3,2003,hummer,h2,base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,2015-06-17
4,2007,ford,fusion,sel,sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,2015-02-03


**Вынесем отдельно колонку с целевым признаком**

In [30]:
target_train = train_mod['sellingprice']

In [31]:
features_train = train_mod.drop(columns=['vin', 'sellingprice'],axis=1)

**Добавим синтетические признаки**

In [32]:
features_train

,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,saledate
0,2011,ford,edge,sel,suv,automatic,md,4.2,111041.0,black,black,santander consumer,2015-06-02
1,2014,ford,fusion,se,sedan,automatic,mo,3.5,31034.0,black,black,ars/avis budget group,2015-02-25
2,2012,nissan,sentra,2.0 sl,sedan,automatic,nj,2.2,35619.0,black,black,nissan-infiniti lt,2015-06-10
3,2003,hummer,h2,base,suv,automatic,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,2015-06-17
4,2007,ford,fusion,sel,sedan,automatic,md,2.0,127709.0,black,black,purple heart,2015-02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440231,2013,kia,optima,ex hybrid,sedan,automatic,ca,4.5,20434.0,white,beige,"kia motors america, inc",2015-01-20
440232,2014,mini,cooper countryman,s all4,wagon,automatic,wi,2.9,14675.0,white,black,bmw mini financial services,2015-02-11
440233,2008,bmw,6 series,650i,coupe,automatic,ca,3.7,102552.0,gray,black,studio city auto group,2015-03-03
440234,2014,ram,2500,slt,crew cab,automatic,tx,3.0,29367.0,white,gray,express car & truck rental,2015-01-14


In [33]:
features_train['make'].value_counts()

ford         74067
chevrolet    47818
nissan       42866
toyota       31711
dodge        24294
             ...  
airstream        1
mazda tk         1
daewoo           1
dot              1
dodge tk         1
Name: make, Length: 63, dtype: int64

In [34]:
def make_duplicates(name):
    if (name=='mercedes-benz') or (name=='mercedes-b'):
        return 'mercedes'
    elif name=='mazda tk':
        return 'mazda'
    elif name == 'vw':
        return 'volkswagen'
    elif name == 'gmc truck':
        return 'gmc'
    elif name == 'ford truck':
        return 'ford'
    elif name == 'airstream':
        return 'mercedes'
    elif name == 'dodge tk':
        return 'dodge'
    else:
        return name

In [35]:
features_train['make']=features_train['make'].apply(make_duplicates)

In [36]:
features_train['make'].unique()

array(['ford', 'nissan', 'hummer', 'lincoln', 'pontiac', 'hyundai', '—',
       'buick', 'chevrolet', 'honda', 'acura', 'cadillac', 'gmc', 'saab',
       'dodge', 'mercedes', 'toyota', 'volkswagen', 'bmw', 'infiniti',
       'chrysler', 'kia', 'jaguar', 'subaru', 'jeep', 'lexus', 'scion',
       'fiat', 'suzuki', 'mazda', 'volvo', 'audi', 'mini', 'isuzu',
       'mitsubishi', 'smart', 'porsche', 'land rover', 'saturn', 'ram',
       'oldsmobile', 'mercury', 'bentley', 'fisker', 'maserati', 'tesla',
       'landrover', 'rolls-royce', 'ferrari', 'plymouth', 'lamborghini',
       'geo', 'aston martin', 'daewoo', 'dot'], dtype=object)

In [37]:
features_train['model'].unique()

array(['edge', 'fusion', 'sentra', 'h2', 'mkz', 'g6', 'escape', 'elantra',
       'freestyle', '—', 'lucerne', 'windstar', 'silverado 1500',
       'murano', 'equinox', 'accord', 'civic', 'mdx', 'cts', 'taurus',
       'srx', 'yukon', '9-3', 'explorer', 'f-150', 'charger', 'armada',
       'gl-class', 'avalon', 'tahoe', 'malibu', 'passat', 'camry', 'flex',
       '3 series', 'q50', 'altima', 'mustang', '300', 'sonata',
       'envoy xl', 'rogue', 'accent', 'maxima', 'forte', 'xf', 'outback',
       'grand caravan', 'e-class', 'cherokee', 'impala', 'optima',
       '5 series', 'expedition', 'is 250', 'mks', '200', 'veloster',
       'golf', 'rav4', 'e300dt', 'five hundred', 'g coupe',
       'grand cherokee', 'x-type', 'g convertible', 'tacoma', 'xa',
       'g sedan', 'tl', 'liberty', 'soul', '500l', 'town and country',
       'hhr', 'wrangler', 'reno', 'suburban', 'pt cruiser', 'gx 460',
       'sienna', 'sl-class', 'envoy', 'patriot', 'e-series van', 'versa',
       'aspen', 'm', 'co

In [38]:
model_counts=pd.DataFrame(features_train['model'].value_counts()).reset_index().rename(
    columns={'index':'model','model':'model_counts'})
model_counts

,model,model_counts
0,altima,15454
1,f-150,11408
2,fusion,10244
3,camry,10000
4,escape,9447
...,...,...
835,alp,1
836,b-series,1
837,pursuit,1
838,tempo,1


In [39]:
features_train=features_train.merge(model_counts, on='model', how='left')

In [40]:
def model_type(row):
    model_counts=row['model_counts']
    model=row['model']
    if model_counts < 1000:
        return '—'
    else:
        return model
features_train['model'] = features_train.apply(model_type, axis=1)
features_train['model'].unique()

array(['edge', 'fusion', 'sentra', '—', 'mkz', 'g6', 'escape', 'elantra',
       'silverado 1500', 'murano', 'equinox', 'accord', 'civic', 'mdx',
       'cts', 'taurus', 'srx', 'yukon', 'explorer', 'f-150', 'charger',
       'tahoe', 'malibu', 'passat', 'camry', '3 series', 'altima',
       'mustang', '300', 'sonata', 'rogue', 'accent', 'maxima', 'forte',
       'outback', 'grand caravan', 'e-class', 'impala', 'optima',
       '5 series', 'expedition', 'is 250', '200', 'rav4', 'g coupe',
       'grand cherokee', 'g sedan', 'tl', 'liberty', 'soul',
       'town and country', 'wrangler', 'suburban', 'sienna', 'patriot',
       'e-series van', 'versa', 'corolla', 'focus', 'traverse',
       'pathfinder', 'silverado 2500hd', 'ram pickup 1500', '4runner',
       'sorento', 'odyssey', 'camaro', 'x5', 'caliber', 'journey',
       'prius', 'jetta', 'avenger', 'cooper', 'f-250 super duty',
       'rx 350', 'cruze', 'm-class', 'c-class', 'cr-v', 'sierra 1500',
       'es 350', 'highlander', 'ran

In [41]:
features_train['trim'].value_counts()

base               43876
se                 34498
lx                 16511
limited            14516
lt                 13431
                   ...  
tsi                    1
4x2 cr limited         1
cruiser touring        1
2.3 premium            1
fwd ext                1
Name: trim, Length: 1851, dtype: int64

In [42]:
trim_counts=pd.DataFrame(features_train['trim'].value_counts()).reset_index().rename(
    columns={'index':'trim','trim':'trim_counts'})
features_train=features_train.merge(trim_counts, on='trim', how='left')
def trim_type(row):
    trim_counts=row['trim_counts']
    trim=row['trim']
    if trim_counts < 1000:
        return '—'
    else:
        return trim
features_train['trim'] = features_train.apply(trim_type, axis=1)

In [43]:
features_train['trim'].unique()

array(['sel', 'se', '—', 'base', 'gls', 'limited', 'ls', 'sl', 'lt', 'lx',
       'luxury', 'denali', 'xlt', 'sxt', 'le', 'titanium', 'sport',
       'work truck', '328i', 'ltz', 'premium', '2.5', 'v6', 'slt', 'sv',
       '3.5 s', '2.5 s', 's', 'lt fleet', 'laredo', 'xle', 'ex',
       'g37 journey', 'xl', 'e-250', '1.8 s', '1lt', 'ls fleet', 'xls',
       'touring', 'sr5', 'g37x', 'sle', 'se pzev', 'r/t', '2lt', 'gt',
       'es', 'ex-l', '3.2', 'lariat', 'st', 'fx4', 'lt1', '+', '3.5 sv',
       'i sport', 'c250 sport', '1.6 sv'], dtype=object)

In [44]:
seller_counts=pd.DataFrame(features_train['seller'].value_counts()).reset_index().rename(
    columns={'index':'seller','seller':'seller_counts'})
features_train=features_train.merge(seller_counts, on='seller', how='left')
def seller_type(row):
    seller_counts=row['seller_counts']
    seller=row['seller']
    if seller_counts < 1000:
        return '—'
    else:
        return seller
features_train['seller'] = features_train.apply(seller_type, axis=1)

In [45]:
features_train['seller'].unique()

array(['santander consumer', 'ars/avis budget group',
       'nissan-infiniti lt', '—', 'ford motor credit company,llc',
       'dt inventory', 'wells fargo dealer services',
       'aaero sweet company', 'avis corporation',
       'ahfc/honda lease trust/hvt, inc. eot', 'wholesale inc # 2',
       'hertz corporation/gdp', 'donlen corporation',
       'ford motor credit company,llc pd',
       'credit acceptance corp/vrs/southfield',
       'toyota financial services', 'ford motor credit company',
       'avis budget group', 'financial services remarketing (lease)',
       'td auto finance', 'kia motors america, inc', 'subaru of america',
       'ge fleet services for itself/servicer', 'the hertz corporation',
       'mercedes-benz financial services', 'gm financial',
       'jpmorgan chase bank,n.a.', 'nissan infiniti lt', 'gm remarketing',
       'hyundai motor finance', 'dt credit corporation',
       'lexus financial services', 'ford-lincoln dealer program',
       'exeter finance 

In [46]:
features_train['year_selling']=pd.DatetimeIndex(features_train['saledate']).year
features_train['month_selling']=pd.DatetimeIndex(features_train['saledate']).month

In [47]:
features_train.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'saledate',
       'model_counts', 'trim_counts', 'seller_counts', 'year_selling',
       'month_selling'],
      dtype='object')

In [48]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440236 entries, 0 to 440235
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   year           440236 non-null  int64         
 1   make           440236 non-null  object        
 2   model          440236 non-null  object        
 3   trim           440236 non-null  object        
 4   body           440236 non-null  object        
 5   transmission   440236 non-null  object        
 6   state          440236 non-null  object        
 7   condition      440236 non-null  float64       
 8   odometer       440236 non-null  float64       
 9   color          440236 non-null  object        
 10  interior       440236 non-null  object        
 11  seller         440236 non-null  object        
 12  saledate       440236 non-null  datetime64[ns]
 13  model_counts   440236 non-null  int64         
 14  trim_counts    440236 non-null  int64         
 15  

In [49]:
features_train=features_train.drop(columns=['model_counts', 'trim_counts', 'seller_counts'])

In [50]:
features_train=features_train.drop(columns=['saledate'])

In [51]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440236 entries, 0 to 440235
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   year           440236 non-null  int64  
 1   make           440236 non-null  object 
 2   model          440236 non-null  object 
 3   trim           440236 non-null  object 
 4   body           440236 non-null  object 
 5   transmission   440236 non-null  object 
 6   state          440236 non-null  object 
 7   condition      440236 non-null  float64
 8   odometer       440236 non-null  float64
 9   color          440236 non-null  object 
 10  interior       440236 non-null  object 
 11  seller         440236 non-null  object 
 12  year_selling   440236 non-null  int64  
 13  month_selling  440236 non-null  int64  
dtypes: float64(2), int64(3), object(9)
memory usage: 50.4+ MB


**кодируем категориальные признаки**

In [52]:
from sklearn.preprocessing import OrdinalEncoder

In [53]:
encoder=OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = 111)

In [54]:
categories = ['make','model', 'trim', 'body','transmission', 'state', 'color', 'interior', 'seller']

In [55]:
encoder.fit(features_train[categories])

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=111)

In [56]:
train_cod = pd.DataFrame(encoder.transform(features_train[categories]), columns=categories)
train_cod

,make,model,trim,body,transmission,state,color,interior,seller
0,15.0,29.0,41.0,39.0,0.0,12.0,1.0,1.0,46.0
1,15.0,42.0,39.0,36.0,0.0,15.0,1.0,1.0,5.0
2,37.0,81.0,58.0,36.0,0.0,19.0,1.0,1.0,42.0
3,19.0,102.0,11.0,39.0,0.0,33.0,6.0,0.0,60.0
4,15.0,42.0,41.0,36.0,0.0,12.0,1.0,1.0,60.0
...,...,...,...,...,...,...,...,...,...
440231,25.0,65.0,58.0,36.0,0.0,3.0,17.0,0.0,33.0
440232,35.0,102.0,58.0,43.0,0.0,37.0,17.0,1.0,60.0
440233,4.0,102.0,58.0,6.0,0.0,3.0,7.0,1.0,60.0
440234,42.0,102.0,44.0,7.0,0.0,33.0,17.0,6.0,60.0


In [57]:
features_keep = ['year', 'condition', 'odometer', 'year_selling', 'month_selling']

In [58]:
scaler=StandardScaler()

In [59]:
scaler.fit(features_train[features_keep])

StandardScaler()

In [60]:
features_scaled =pd.DataFrame(scaler.transform(features_train[features_keep]), columns = features_keep)

In [61]:
feat_train_mod = pd.concat([features_scaled, train_cod], axis=1)

In [62]:
feat_train_mod

,year,condition,odometer,year_selling,month_selling,make,model,trim,body,transmission,state,color,interior,seller
0,0.241305,0.820315,0.797543,0.328663,0.672419,15.0,29.0,41.0,39.0,0.0,12.0,1.0,1.0,46.0
1,0.995464,0.075721,-0.696844,0.328663,-0.560021,15.0,42.0,39.0,36.0,0.0,15.0,1.0,1.0,5.0
2,0.492692,-1.307095,-0.611205,0.328663,0.672419,37.0,81.0,58.0,36.0,0.0,19.0,1.0,1.0,42.0
3,-1.769785,-0.668872,1.175963,0.328663,0.672419,19.0,102.0,11.0,39.0,0.0,33.0,6.0,0.0,60.0
4,-0.764240,-1.519836,1.108871,0.328663,-0.560021,15.0,42.0,41.0,36.0,0.0,12.0,1.0,1.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440231,0.744078,1.139426,-0.894833,0.328663,-0.868131,25.0,65.0,58.0,36.0,0.0,3.0,17.0,0.0,33.0
440232,0.995464,-0.562502,-1.002401,0.328663,-0.560021,35.0,102.0,58.0,43.0,0.0,37.0,17.0,1.0,60.0
440233,-0.512854,0.288462,0.638984,0.328663,-0.251911,4.0,102.0,58.0,6.0,0.0,3.0,7.0,1.0,60.0
440234,0.995464,-0.456131,-0.727981,0.328663,-0.868131,42.0,102.0,44.0,7.0,0.0,33.0,17.0,6.0,60.0


In [63]:
target_train

0         12500
1         14500
2          9100
3         13300
4          1300
          ...  
440231    16000
440232    24250
440233    16750
440234    33000
440235    14500
Name: sellingprice, Length: 440236, dtype: int64

In [64]:
model = RandomForestRegressor(n_estimators=10)

In [65]:
scores=cross_val_score(model,feat_train_mod,target_train, scoring = 'neg_mean_absolute_percentage_error', cv=5)

In [66]:
scores

array([-0.21231614, -0.24236718, -0.21122337, -0.20960962, -0.25214753])

In [67]:
scores.mean()

-0.22553276613408602

Значение метрики отрицательно из-за того, что в cross-val-score доступно только neg_mean_absolute_percentage_error по причине "Thus metrics which measure the distance between the model and the data, like metrics.mean_squared_error, are available as neg_mean_squared_error which return the negated value of the metric." (см https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter )

Адекватным значением метрики MAPE считается менее 20%. 
**Попробуем улучшить модель**.

**Также можно пробовать другие типы кодирования категорий**

In [68]:
model.fit(feat_train_mod,target_train)

RandomForestRegressor(n_estimators=10)

**Предскажем тестовые**

**сначала преобразуем данные по алгоритму для train**

In [69]:
features_test = test.drop(columns=['vin'],axis=1)

In [82]:
features_test['condition']=features_test['condition'].fillna(median_condition)
features_test['odometer']=features_test['odometer'].fillna(median_odometer)
features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110058 non-null  int64  
 1   make          107997 non-null  object 
 2   model         107979 non-null  object 
 3   trim          107944 non-null  object 
 4   body          107464 non-null  object 
 5   transmission  97047 non-null   object 
 6   state         110058 non-null  object 
 7   condition     110058 non-null  float64
 8   odometer      110058 non-null  float64
 9   color         109900 non-null  object 
 10  interior      109900 non-null  object 
 11  seller        110058 non-null  object 
 12  saledate      110058 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 10.9+ MB


In [83]:
features_test['make'].unique()

array(['Cadillac', 'GMC', 'Nissan', 'Chevrolet', 'Volkswagen', 'Hyundai',
       'Ford', 'Infiniti', 'Subaru', 'Kia', 'BMW', nan, 'Mitsubishi',
       'Dodge', 'Oldsmobile', 'Volvo', 'Pontiac', 'Acura', 'Honda',
       'Toyota', 'Audi', 'Jeep', 'Lincoln', 'Chrysler', 'Mercedes-Benz',
       'Ram', 'Scion', 'Mercury', 'Lexus', 'Saturn', 'Mazda', 'Porsche',
       'MINI', 'smart', 'Buick', 'HUMMER', 'ford', 'chrysler', 'Jaguar',
       'mitsubishi', 'FIAT', 'Saab', 'mazda', 'Suzuki', 'Land Rover',
       'bmw', 'cadillac', 'oldsmobile', 'honda', 'acura', 'buick',
       'landrover', 'Geo', 'lexus', 'dodge', 'chevrolet', 'chev truck',
       'nissan', 'Bentley', 'jeep', 'mercury', 'toyota', 'gmc', 'Isuzu',
       'Aston Martin', 'Maserati', 'land rover', 'kia', 'subaru',
       'Daewoo', 'volkswagen', 'Tesla', 'mercedes', 'Ferrari', 'hyundai',
       'pontiac', 'Fisker', 'porsche', 'Rolls-Royce', 'Lotus', 'Plymouth',
       'hyundai tk', 'lincoln', 'ford tk', 'audi'], dtype=object)

In [85]:
lower_columns = ['make', 'model', 'trim', 'body', 'transmission','state', 'color', 'interior', 'seller']
for i in lower_columns:
    features_test[i]=features_test[i].str.lower()
features_test['make'].unique()

array(['cadillac', 'gmc', 'nissan', 'chevrolet', 'volkswagen', 'hyundai',
       'ford', 'infiniti', 'subaru', 'kia', 'bmw', nan, 'mitsubishi',
       'dodge', 'oldsmobile', 'volvo', 'pontiac', 'acura', 'honda',
       'toyota', 'audi', 'jeep', 'lincoln', 'chrysler', 'mercedes-benz',
       'ram', 'scion', 'mercury', 'lexus', 'saturn', 'mazda', 'porsche',
       'mini', 'smart', 'buick', 'hummer', 'jaguar', 'fiat', 'saab',
       'suzuki', 'land rover', 'landrover', 'geo', 'chev truck',
       'bentley', 'isuzu', 'aston martin', 'maserati', 'daewoo', 'tesla',
       'mercedes', 'ferrari', 'fisker', 'rolls-royce', 'lotus',
       'plymouth', 'hyundai tk', 'ford tk'], dtype=object)

In [87]:
features_test['saledate']=pd.to_datetime(features_test['saledate'], format='%a %b %d %Y', exact=False)

In [88]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   year          110058 non-null  int64         
 1   make          110058 non-null  object        
 2   model         110058 non-null  object        
 3   trim          110058 non-null  object        
 4   body          110058 non-null  object        
 5   transmission  110058 non-null  object        
 6   state         110058 non-null  object        
 7   condition     110058 non-null  float64       
 8   odometer      110058 non-null  float64       
 9   color         110058 non-null  object        
 10  interior      110058 non-null  object        
 11  seller        110058 non-null  object        
 12  saledate      110058 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(9)
memory usage: 10.9+ MB


**Добавим синтетические признаки**

In [89]:
features_test.head(5)

,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,saledate
0,2005,cadillac,cts,base,sedan,automatic,ca,2.7,116970.0,silver,black,lexus of stevens creek,2015-01-14
1,2014,gmc,savana cargo,2500,van,—,pa,4.4,6286.0,white,gray,u-haul,2015-02-27
2,2013,nissan,murano,s,suv,automatic,oh,4.6,11831.0,gray,black,nissan-infiniti lt,2015-02-24
3,2013,chevrolet,impala,ls fleet,sedan,automatic,fl,2.3,57105.0,silver,black,onemain rem/auto club of miami inc dba north dad,2015-03-06
4,2013,nissan,titan,sv,crew cab,automatic,tn,2.9,31083.0,black,black,nissan north america inc.,2015-06-03


In [91]:
features_test['make']=features_test['make'].apply(make_duplicates)

In [92]:
features_test['make'].unique()

array(['cadillac', 'gmc', 'nissan', 'chevrolet', 'volkswagen', 'hyundai',
       'ford', 'infiniti', 'subaru', 'kia', 'bmw', '—', 'mitsubishi',
       'dodge', 'oldsmobile', 'volvo', 'pontiac', 'acura', 'honda',
       'toyota', 'audi', 'jeep', 'lincoln', 'chrysler', 'mercedes', 'ram',
       'scion', 'mercury', 'lexus', 'saturn', 'mazda', 'porsche', 'mini',
       'smart', 'buick', 'hummer', 'jaguar', 'fiat', 'saab', 'suzuki',
       'land rover', 'landrover', 'geo', 'chev truck', 'bentley', 'isuzu',
       'aston martin', 'maserati', 'daewoo', 'tesla', 'ferrari', 'fisker',
       'rolls-royce', 'lotus', 'plymouth', 'hyundai tk', 'ford tk'],
      dtype=object)

In [93]:
model_counts=pd.DataFrame(features_test['model'].value_counts()).reset_index().rename(
    columns={'index':'model','model':'model_counts'})
model_counts

,model,model_counts
0,altima,3736
1,f-150,2737
2,fusion,2553
3,camry,2423
4,escape,2296
...,...,...
742,aspire,1
743,e-150,1
744,s55,1
745,f-150 svt lightning,1


In [94]:
features_test=features_test.merge(model_counts, on='model', how='left')

In [95]:
def model_type(row):
    model_counts=row['model_counts']
    model=row['model']
    if model_counts < 1000:
        return '—'
    else:
        return model
features_test['model'] = features_test.apply(model_type, axis=1)
features_test['model'].unique()

array(['—', 'impala', 'sonata', 'explorer', 'g sedan', 'escape',
       'silverado 1500', 'grand caravan', 'focus', 'fusion', 'accord',
       'maxima', 'camry', 'corolla', 'f-150', 'elantra', 'altima',
       'town and country', 'civic', 'c-class', 'cruze', 'malibu',
       '3 series', 'mustang', 'sentra', 'edge', 'rogue'], dtype=object)

In [96]:
features_test['trim'].value_counts()

base                     11009
se                        8725
lx                        4098
limited                   3536
lt                        3280
                         ...  
5.2 quattro                  1
sport s                      1
3500 high roof 158 wb        1
gt limited edition           1
rover 4.2l s/c               1
Name: trim, Length: 1458, dtype: int64

In [97]:
trim_counts=pd.DataFrame(features_test['trim'].value_counts()).reset_index().rename(
    columns={'index':'trim','trim':'trim_counts'})
features_test=features_test.merge(trim_counts, on='trim', how='left')
def trim_type(row):
    trim_counts=row['trim_counts']
    trim=row['trim']
    if trim_counts < 1000:
        return '—'
    else:
        return trim
features_test['trim'] = features_test.apply(trim_type, axis=1)

In [98]:
features_test['trim'].unique()

array(['base', '—', 's', 'sv', 'gls', 'lt', 'se', 'sel', 'lx', 'ls', 'le',
       'ex-l', 'sport', '2.5 s', 'limited', 'sxt', 'xlt', 'ex', 'ltz',
       'slt', 'touring', '2.5'], dtype=object)

In [99]:
seller_counts=pd.DataFrame(features_test['seller'].value_counts()).reset_index().rename(
    columns={'index':'seller','seller':'seller_counts'})
features_test=features_test.merge(seller_counts, on='seller', how='left')
def seller_type(row):
    seller_counts=row['seller_counts']
    seller=row['seller']
    if seller_counts < 1000:
        return '—'
    else:
        return seller
features_test['seller'] = features_test.apply(seller_type, axis=1)

In [100]:
features_test['seller'].unique()

array(['—', 'nissan-infiniti lt', 'ge fleet services for itself/servicer',
       'nissan infiniti lt', 'jpmorgan chase bank,n.a.',
       'enterprise veh exchange/rental', 'hyundai motor finance',
       'tdaf remarketing', 'ford motor credit company,llc',
       'avis corporation', 'ahfc/honda lease trust/hvt, inc. eot',
       'wells fargo dealer services', 'santander consumer',
       'the hertz corporation', 'financial services remarketing (lease)',
       'gm financial', 'chrysler capital',
       'mercedes-benz financial services'], dtype=object)

In [101]:
features_test['year_selling']=pd.DatetimeIndex(features_test['saledate']).year
features_test['month_selling']=pd.DatetimeIndex(features_test['saledate']).month

In [102]:
features_test.columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'saledate',
       'model_counts', 'trim_counts', 'seller_counts', 'year_selling',
       'month_selling'],
      dtype='object')

In [103]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110058 entries, 0 to 110057
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   year           110058 non-null  int64         
 1   make           110058 non-null  object        
 2   model          110058 non-null  object        
 3   trim           110058 non-null  object        
 4   body           110058 non-null  object        
 5   transmission   110058 non-null  object        
 6   state          110058 non-null  object        
 7   condition      110058 non-null  float64       
 8   odometer       110058 non-null  float64       
 9   color          110058 non-null  object        
 10  interior       110058 non-null  object        
 11  seller         110058 non-null  object        
 12  saledate       110058 non-null  datetime64[ns]
 13  model_counts   110058 non-null  int64         
 14  trim_counts    110058 non-null  int64         
 15  

In [104]:
features_test=features_test.drop(columns=['model_counts', 'trim_counts', 'seller_counts', 'saledate'])

In [106]:
test_cod = pd.DataFrame(encoder.transform(features_test[categories]), columns=categories)
test_cod

,make,model,trim,body,transmission,state,color,interior,seller
0,6.0,102.0,11.0,36.0,0.0,3.0,15.0,1.0,60.0
1,17.0,102.0,58.0,42.0,2.0,28.0,17.0,6.0,60.0
2,37.0,102.0,38.0,39.0,0.0,24.0,7.0,1.0,42.0
3,7.0,49.0,58.0,36.0,0.0,5.0,15.0,1.0,60.0
4,37.0,102.0,48.0,7.0,0.0,32.0,1.0,1.0,60.0
...,...,...,...,...,...,...,...,...,...
110053,24.0,102.0,45.0,39.0,0.0,3.0,11.0,1.0,60.0
110054,29.0,102.0,11.0,36.0,0.0,5.0,7.0,1.0,60.0
110055,37.0,57.0,58.0,36.0,0.0,35.0,15.0,6.0,60.0
110056,37.0,102.0,11.0,6.0,1.0,33.0,19.0,1.0,60.0


In [107]:
features_keep = ['year', 'condition', 'odometer', 'year_selling', 'month_selling']

In [108]:
features_test_scaled =pd.DataFrame(scaler.transform(features_test[features_keep]), columns = features_keep)

In [109]:
feat_test_mod = pd.concat([features_test_scaled, test_cod], axis=1)
feat_test_mod

,year,condition,odometer,year_selling,month_selling,make,model,trim,body,transmission,state,color,interior,seller
0,-1.267013,-0.775243,0.908286,0.328663,-0.868131,6.0,102.0,11.0,36.0,0.0,3.0,15.0,1.0,60.0
1,0.995464,1.033056,-1.159092,0.328663,-0.560021,17.0,102.0,58.0,42.0,2.0,28.0,17.0,6.0,60.0
2,0.744078,1.245797,-1.055522,0.328663,-0.560021,37.0,102.0,38.0,39.0,0.0,24.0,7.0,1.0,42.0
3,0.744078,-1.200725,-0.209885,0.328663,-0.251911,7.0,49.0,58.0,36.0,0.0,5.0,15.0,1.0,60.0
4,0.744078,-0.562502,-0.695929,0.328663,0.672419,37.0,102.0,48.0,7.0,0.0,32.0,1.0,1.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110053,0.241305,0.394833,-0.042154,0.328663,-0.868131,24.0,102.0,45.0,39.0,0.0,3.0,11.0,1.0,60.0
110054,0.995464,1.458538,-0.947991,0.328663,0.672419,29.0,102.0,11.0,36.0,0.0,5.0,7.0,1.0,60.0
110055,-1.518399,-0.562502,1.040266,0.328663,-0.560021,37.0,57.0,58.0,36.0,0.0,35.0,15.0,6.0,60.0
110056,0.744078,-1.307095,-0.769035,0.328663,0.672419,37.0,102.0,11.0,6.0,1.0,33.0,19.0,1.0,60.0


In [110]:
feat_test_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   year           110058 non-null  float64
 1   condition      110058 non-null  float64
 2   odometer       110058 non-null  float64
 3   year_selling   110058 non-null  float64
 4   month_selling  110058 non-null  float64
 5   make           110058 non-null  float64
 6   model          110058 non-null  float64
 7   trim           110058 non-null  float64
 8   body           110058 non-null  float64
 9   transmission   110058 non-null  float64
 10  state          110058 non-null  float64
 11  color          110058 non-null  float64
 12  interior       110058 non-null  float64
 13  seller         110058 non-null  float64
dtypes: float64(14)
memory usage: 11.8 MB


**Теперь сами предсказания и сохранение итогового файла**

In [111]:
predicted_prices = pd.DataFrame(model.predict(feat_test_mod), columns = ['sellingprice'])

In [112]:
predicted_prices

,sellingprice
0,3850.0
1,22320.0
2,16830.0
3,11330.0
4,20490.0
...,...
110053,13590.2
110054,43315.0
110055,4105.0
110056,21010.0


In [113]:
output_table = pd.concat([test['vin'],predicted_prices], axis=1)

In [114]:
output_table

,vin,sellingprice
0,1g6dp567450124779,3850.0
1,1gtw7fca7e1902207,22320.0
2,jn8az1mw6dw303497,16830.0
3,2g1wf5e34d1160703,11330.0
4,1n6aa0ec3dn301209,20490.0
...,...,...
110053,1j4aa2d17bl584330,13590.2
110054,jthbf1d23e5007526,43315.0
110055,1n4ba41e54c831950,4105.0
110056,jn1az4eh3dm382431,21010.0


In [115]:
output_table.to_csv('second_prediction.csv', index=False)